# 1. Mapa de Casos Encontrados

## Importações

In [2]:
import pandas as pd
import pydeck as pdk

### Lê o ``.csv``

Estações

In [4]:
cols=[
    'Nome do Município', 'Estado', 'Nome da Estação', 'Latitude', 'Longitude'
]

stations_df = pd.read_csv("../data/monitor_ar/EstacoesMonitorAr-Nov-2022.csv", sep=";", usecols=cols)


### Ajusta a variável ``coords``

In [ ]:
stations = stations_df.rename(columns = {
    "Nome do Município": "Nome_Municipio",
    "Nome da Estação": "Nome_Estacao"
})

stations['Latitude'] = stations['Latitude'].str.replace(',','.')
stations['Longitude'] = stations['Longitude'].str.replace(',','.')

# Converte lat/lon para numérico, tratando erros
stations['Latitude'] = pd.to_numeric(stations['Latitude'], errors='coerce')
stations['Longitude'] = pd.to_numeric(stations['Longitude'], errors='coerce')
stations.dropna(subset=['Latitude', 'Longitude'], inplace=True)

stations

## 2. Exibir o Mapa de Estações

In [ ]:
def render_map(df):
    layer = pdk.Layer(
        "ScatterplotLayer",
        df,
        pickable=True,
        opacity=0.7,
        stroked=True,
        filled=True,
        radius_scale=6,
        radius_min_pixels=5,
        radius_max_pixels=5,
        line_width_min_pixels=1,
        get_position="coordinates",
        get_fill_color=[0, 158, 96],
        get_line_color=[0, 0, 0],
    )

    view_state = pdk.ViewState(latitude=-23, longitude=-50, zoom=3.5, bearing=0, pitch=0)

    html_tooltip = """
        <div style="
            background-color: rgba(255, 255, 255, 0.9); 
            border: 1px solid #d1d1d1;
            border-radius: 8px; 
            padding: 12px; 
            color: #333; 
            font-family: Arial, sans-serif;
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            max-width: 300px;
        ">
            <div style="font-size: 16px; font-weight: bold; margin-bottom: 5px; color: #0050ff;">
                {nome_estacao}
            </div>
            
            <div style="margin-bottom: 10px; font-size: 13px;">
                {nome_municipio} - {estado}
            </div>

            <hr style="border: none; border-top: 1px solid #eee; margin: 10px 0;">
        </div>
        """

    r = pdk.Deck(
        layers=[layer],
        initial_view_state=view_state,
        tooltip={
            "text": "Estação: {Nome_Estacao}\n{Nome_Municipio} - {Estado}"
        },
    )

    r.picking_radius = 100

    display(r)

print("Colunas disponíveis no DataFrame 'stations':")
print(stations.columns.tolist())

render_map(stations)